# MOwNiT - Generatory liczb losowych

## Problem logarytmu dyskretnego

Niech $(G,\circ)$ będzie grupą z działaniem $\circ$ i elementem neutralnym $1_G$. Wtedy dla dowolnego elementu $a\in G$ i $k\in\mathbb{Z}$ definiujemy *potęgę* $$a^k =\left\{\begin{array}{cc}
\underbrace{a\circ a\circ \ldots \circ a}_{k}&\text{ dla }k>0\\
1_G&\text{ dla }k=0\\
\underbrace{a^{-1}\circ a^{-1}\circ \ldots \circ a^{-1}}_{k}&\text{ dla }k<0
\end{array}\right.$$
gdzie $a^{-1}$ jest elementem odwrotnym do $a$.

Dla $a,b\in G$, $b\neq 1_G$, *logarytmem dyskretnym* $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a$.

### Logarytm dyskretny w $\mathbb{Z}_n$

W przypadku pierścienia $\mathbb{Z}_n$ logarytmem dyskretnym $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a\mod n$, o ile w ogóle istnieje.

Specyficzną sytuacją w teorii liczb jest gdy $n=p$ jest liczbą pierwszą a $q$ jest pierwiastkiem pierwotnym $\mod p$. Wtedy:
- potęgi $q^k\mod p$ generują cały zbiór $[1,p-1]$, tzn. $q$ jest generatorem grupy multiplikatywnej rzędu $p-1$
- logarytm dyskretny $\log_q a$ istnieje dla każdego niezerowego elementu $a\in \mathbb{Z}_p$

## Algorytm wymiany klucza Diffiego-Hellmana

Alice i Bob uzgadniają klucz publiczny będący liczbą pierwszą $p$ oraz $q$ - pierwiastkiem pierwotnym mod $p$.
- sekret Alice: liczba całkowita $n\in \mathbb{Z}_p\setminus\{0\}$ losowa, z rozkładu jednostajnego
- sekret Boba: liczba całkowita $m\in \mathbb{Z}_p\setminus\{0\}$ losowa, z rozkładu jednostajnego
- Alice generuje $x=q^n\mod p$ i wysyła do Boba
- Bob generuje $y=q^m\mod p$ i wysyła Alice
- Alice oblicza klucz $k=y^n\mod p$
- Bob oblicza klucz $k=x^m\mod p$


## Zadanie 1. (2 pkt)

Zaimplementuj wybrany algorytm generowania liczb losowych z rozkładem jednostajnym w pierścieniu $\mathbb{Z}_p$.

In [1]:
import time

class LCG:
    """
    Prosty liniowo‐kongruencyjny generator:
        X_{n+1} = (a * X_n + c) mod m
    z parametrami domyślnymi zgodnymi z glibc.
    """
    def __init__(self, seed: int = None,
                 a: int = 1103515245,
                 c: int = 12345,
                 m: int = 2**31):
        self.m = m
        self.a = a
        self.c = c
        if seed is None:
            seed = time.time_ns() % m
        self.state = seed

    def next(self) -> int:
        """Wylicz kolejny stan i zwróć go."""
        self.state = (self.a * self.state + self.c) % self.m
        return self.state

    def rand_zp(self, p: int) -> int:
        """
        Zwraca jednolicie losowy element z Z_p^* = {1,2,...,p−1}.
        """
        return (self.next() % (p - 1)) + 1


# === Test losowości LCG ===
p = 23

print("=== Test z powtarzalnym ziarnem ===")
rng1 = LCG(seed=123)
print("rng1, seed=123:", [rng1.rand_zp(p) for _ in range(10)])

rng1b = LCG(seed=123)
print("rng1b, seed=123:", [rng1b.rand_zp(p) for _ in range(10)])

print("\n=== Test z ziarnem opartym na czasie ===")
rng2 = LCG()  # time.time_ns()
time.sleep(0.001)  # odczekaj 1 ms
rng3 = LCG()

print("rng2, time-based seed:", [rng2.rand_zp(p) for _ in range(10)])
print("rng3, time-based seed:", [rng3.rand_zp(p) for _ in range(10)])

=== Test z powtarzalnym ziarnem ===
rng1, seed=123: [15, 20, 11, 4, 7, 12, 1, 6, 1, 8]
rng1b, seed=123: [15, 20, 11, 4, 7, 12, 1, 6, 1, 8]

=== Test z ziarnem opartym na czasie ===
rng2, time-based seed: [2, 9, 6, 17, 18, 11, 16, 13, 2, 5]
rng3, time-based seed: [6, 15, 12, 5, 12, 15, 4, 11, 22, 19]


## Zadanie 2. (2 pkt)

Wykorzystaj algorytm z zadania pierwszego do implementacji wymiany klucza Diffie-Hellman.

In [2]:
def diffie_hellman(p: int, q: int, rng: LCG):
    # Alice
    a = rng.rand_zp(p)
    A = pow(q, a, p)
    # Bob
    b = rng.rand_zp(p)
    B = pow(q, b, p)
    # klucze obliczone przez Alice i Boba
    k_a = pow(B, a, p)
    k_b = pow(A, b, p)
    return a, b, A, B, k_a, k_b

# -- Wykonujemy dwie sesje z różnymi ziarnami na większym p --
p, q = 10007, 5       # p to losowa liczba pierwsza ~10k, q dowolny generator
results = []
for seed in (100, 200):
    rng = LCG(seed=seed)
    a, b, A, B, k_a, k_b = diffie_hellman(p, q, rng)
    results.append((seed, a, b, A, B, k_a, k_b))


for seed, a, b, A, B, k_a, k_b in results:
    print(f"--- Sesja (seed={seed}) ---")
    print(f"Alice secret: {a}")
    print(f"Bob   secret: {b}")
    print(f"Alice pub   : {A}")
    print(f"Bob   pub   : {B}")
    print(f"Key by Alice: {k_a}")
    print(f"Key by Bob  : {k_b}")
    print("Keys match? :", k_a == k_b, "\n")


--- Sesja (seed=100) ---
Alice secret: 3176
Bob   secret: 5339
Alice pub   : 9948
Bob   pub   : 918
Key by Alice: 8328
Key by Bob  : 8328
Keys match? : True 

--- Sesja (seed=200) ---
Alice secret: 4012
Bob   secret: 1719
Alice pub   : 1033
Bob   pub   : 8682
Key by Alice: 2106
Key by Bob  : 2106
Keys match? : True 



## Zadanie 3. (1 pkt)
Na swoją własną implementację przeprowadź atak metodą brute force. Jeżeli zakończy się sukcesem - zmień parametry kryptosystemu tak, by atak brute force nie przyniósł rezultatów w rozsądnym czasie (rozsądny czas = 10 minut).

In [3]:
import time

# 1. LCG – prosty generator liczb pseudolosowych
class LCG:
    def __init__(self, seed=None,
                 a=1103515245, c=12345, m=2 ** 31):
        self.a, self.c, self.m = a, c, m
        self.state = seed if seed is not None else int(time.time() * 1000) % m

    def next(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state

    def rand_zp(self, p):
        return (self.next() % (p - 1)) + 1


# 2. Diffie–Hellman – z wymuszonym dużym sekretami
def diffie_hellman(p: int, q: int, rng: LCG):
    # Alice
    a = rng.rand_zp(p)
    A = pow(q, a, p)
    # Bob
    b = rng.rand_zp(p)
    B = pow(q, b, p)
    # klucze obliczone przez Alice i Boba
    k_a = pow(B, a, p)
    k_b = pow(A, b, p)
    return a, b, A, B, k_a, k_b


# 3. Brute-force pełny – tylko do bardzo małych p
def brute_force_dlog(p, g, y):
    start = time.time()
    for x in range(1, p):
        if pow(g, x, p) == y:
            return x, time.time() - start
    return None, time.time() - start


# 4. Brute-force ograniczony czasowo – benchmark
def timed_brute_force_attempt(p, g, y, duration_sec=5):
    print(f"\n[Benchmark brute-force {duration_sec}s  (p ≈ {p.bit_length()} bit ≈ {p:,})]")
    start = time.time()
    count = 0
    x = 1
    while time.time() - start < duration_sec:
        if pow(g, x, p) == y:
            t = time.time() - start
            print(f"‣ ***Znaleziono sekret: x = {x}  w {t:.3f}s***")
            return x, t
        x += 1
        count += 1

    elapsed = time.time() - start or 1.0
    ips = count / elapsed
    est_total = (p - 1) / ips
    est_min = est_total / 60
    print(f"‣ Nie znaleziono w {elapsed:.2f}s – sprawdzono {count:,} kandydata(ów)")
    print(f"‣ Szybkość ≈ {ips:,.0f} prób/s")
    print(f"‣ Szacowany czas pełnego brute-force: {est_min:,.1f} minut\n")
    return None, elapsed


# 5. Demonstracja – małe i duże p
def demo():
    # 5a. Małe p (pokaz złamania w sekundach)
    print("═" * 60)
    print("● ATAK na BARDZO MAŁYM p")
    p_small = 10_007
    g_small = 5
    rng = LCG(seed=123)
    a, b, A, B, kA, kB = diffie_hellman(p_small, g_small, rng)

    print(f"p = {p_small}, g = {g_small}")
    print(f"Alice publiczny A = {A}")
    secret, t = brute_force_dlog(p_small, g_small, A)
    print(f"‣ Złamano sekret w {t:.4f}s →  x = {secret}\n")

    # 5b. Większe p ≈ 900 mln (benchmark brute-force)
    print("═" * 60)
    print("● MODUŁ na ~900 MLN – benchmark")
    P_BIG = 900001117    # liczba pierwsza ~120 mln
    G_BIG = 2

    rng = LCG(seed=42)
    a, b, A, B, kA, kB = diffie_hellman(P_BIG, G_BIG, rng)
    print(f"p = {P_BIG:,} (bit length = {P_BIG.bit_length()})")
    print(f"Alice publiczny A = {A}")
    timed_brute_force_attempt(P_BIG, G_BIG, A, duration_sec=5)

    print("═" * 60)
    print("Wniosek: ten moduł wymaga brute-force trwającego znacznie powyżej 5 sekund,\n"
          "Nie jest on praktycznie bezpieczny, ale tutaj realnie pokazuje, ze czas się wydłuza\n")


if __name__ == "__main__":
    demo()

════════════════════════════════════════════════════════════
● ATAK na BARDZO MAŁYM p
p = 10007, g = 5
Alice publiczny A = 2169
‣ Złamano sekret w 0.0021s →  x = 3267

════════════════════════════════════════════════════════════
● MODUŁ na ~900 MLN – benchmark
p = 900,001,117 (bit length = 30)
Alice publiczny A = 153888958

[Benchmark brute-force 5s  (p ≈ 30 bit ≈ 900,001,117)]
‣ Nie znaleziono w 5.00s – sprawdzono 6,129,612 kandydata(ów)
‣ Szybkość ≈ 1,225,922 prób/s
‣ Szacowany czas pełnego brute-force: 12.2 minut

════════════════════════════════════════════════════════════
Wniosek: ten moduł wymaga brute-force trwającego znacznie powyżej 5 sekund,
Nie jest on praktycznie bezpieczny, ale tutaj realnie pokazuje, ze czas się wydłuza

